**Install Packages**

In [ ]:
!pip install torch trochvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement trochvision (from versions: none)
ERROR: No matching distribution found for trochvision


In [ ]:
!pip install laserembeddings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 KB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 6.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=fb9817ca01a58aee3b6cc56b44b8fbf769660231f8ff08e31eef9dff388f603d
  Stored in directory: /root/.cache/pip/wheels/c4/df/30/3d6c623db99d503dcdbae1f686953b7c1a0754d8a658dc0845
Successfully built sacremoses


In [ ]:
import pandas as pd
import random as rd
import json
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors


In [ ]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


# Monolingual Embedding

> Indented block



In [ ]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['به ملازمان سلطان که رساند این دعا را که به شکر پادشاهی ز نظر مران گدا را ز رقیب دیوسیرت به خدای خود پناهم مگر آن شهاب ثاقب مددی دهد خدا را',
     'درخت دوستی بنشان که کام دل به بار آرد'],
    lang='fa')  # lang is only used for tokenization

In [ ]:
print(embeddings.shape)
print(embeddings)
%cd "/content/"

(2, 1024)
[[ 0.01072003  0.00345858  0.00174004 ...  0.02418301  0.01499301
   0.01545107]
 [ 0.0038445   0.00011095  0.00274064 ...  0.0077541   0.00159712
  -0.00810177]]
/content


# Multilingual Embedding

In [ ]:
#Example
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.'],
    lang=['en', 'fr', 'de'])

**Input data set**

read input data set and creat the input table

In [ ]:
data_set_root = "/content/drive/MyDrive/Text_enrich/datasets/"
input_data = pd.read_csv(data_set_root + "input_text/input_text.csv")

In [ ]:
columns = ["input_text" , "vector"]
input_table = pd.DataFrame([], columns=columns)
for sample in rd.sample(list(pd.unique(input_data["input_text"])),100):
  input_table = input_table.append({'input_text':sample,'vector':laser.embed_sentences(sample, 'fa')}, ignore_index = True)

In [ ]:
input_table

,input_text,vector
0,باید حقوق حیوانات را رعایت کنیم,"[[0.0018938975, -6.682478e-05, 0.0017122207, 0..."
1,نرسیدن به اهداف نباید باعث ناراحتی شود,"[[0.0050546504, 0.0023843201, 0.0010176041, 0...."
2,کنترل نکردن زبان انسان را بی‌ارزش می‌کند.,"[[0.005944965, -0.00010063578, 0.0004112802, 0..."
3,انسان واقعی در برابر ظلم سکوت نمی‌کند,"[[0.0007534189, -0.0004926382, 0.02172059, 0.0..."
4,باید به حقوق همسایگان خود احترام گذاشت,"[[0.0054228613, -0.000264998, 0.002452892, 0.0..."
...,...,...
95,بنده دیگران شدن کار اشتباهی است,"[[0.04188226, -0.00043034053, 0.016514702, 0.0..."
96,مردم را باید از کارهای بد باز داشت,"[[0.0061172238, -0.0001422685, -0.00018344144,..."
97,انسان باید خود را گرامی بدارد,"[[0.0038419664, -3.422694e-05, 0.04811488, 0.0..."
98,صبر یکی از صفات خردمندان است,"[[0.014067072, -0.00011483547, -0.0008464994, ..."


# Output data set embedding




English qoutes

creating lightweight data set

In [ ]:
# columns = ["output_text" , "vector", "output_data_source","index"]
# English_qoute_table = pd.DataFrame([], columns=columns)
# f = open(data_set_root +'quotes/quotes.jsonl')
# new_file = open(data_set_root + "quotes/light_quotes.jsonl", "w")

# quotes = [json.loads(jline) for jline in f.read().splitlines()]
# print(quotes[0])
# n = 0
import pandas as pd    
jsonObj = pd.read_json(path_or_buf= data_set_root +'quotes/quotes.jsonl', lines=True)

for i in range(0, len(jsonObj)):
  if len(jsonObj["quote"][i]) > 210:
    jsonObj = jsonObj.drop([i])
    
with open(data_set_root + "quotes/light_quotes.jsonl", "w") as f:
    f.write(jsonObj.to_json(orient='records', lines=True, force_ascii=False))
# for i in range(len(quotes)):
#   if 20 < len(quotes[i]["quote"]) < 210:
#     new_file.write(str(quotes[i])+ '\n')
#     n+=1
# new_file.close()
# f.close()

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
English_qoute_table = pd.DataFrame([], columns=columns)
f = open(data_set_root +'quotes/light_quotes.jsonl')
quotes = [json.loads(jline) for jline in f.read().splitlines()]
n = 0
for i in range(len(quotes)):
  if len(quotes[i]["quote"]) < 210:
    n+=1
    English_qoute_table = English_qoute_table.append({'output_text':quotes[i]["quote"][1:-2],'vector':laser.embed_sentences(quotes[i]["quote"][1:-2], 'en'), 'output_data_source':'نقل قول از نویسندگان انگلیسی', 'index':n}, ignore_index = True)

f.close()

In [ ]:
pd.DataFrame(English_qoute_table).to_csv(data_set_root+ "LASER_output/light_quotes.csv")

In [ ]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(English_qoute_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(English_qoute_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + English_qoute_table["output_text"].iloc[index] )  
  print("_"*100) 
     

باید حقوق حیوانات را رعایت کنیم => The greatness of a nation and its moral progress can be judged by the way its animals are treated
____________________________________________________________________________________________________
نرسیدن به اهداف نباید باعث ناراحتی شود => Don't bend; don't water it down; don't try to make it logical; don't edit your own soul according to the fashion. Rather, follow your most intense obsessions mercilessly
____________________________________________________________________________________________________
کنترل نکردن زبان انسان را بی‌ارزش می‌کند. => A man can no more diminish God's glory by refusing to worship Him than a lunatic can put out the sun by scribbling the word 'darkness' on the walls of his cell
____________________________________________________________________________________________________
انسان واقعی در برابر ظلم سکوت نمی‌کند => Never be afraid to raise your voice for honesty and truth and compassion against injustice and lying and g

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
Nahj_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'nahj/Nahj Al-Balaqa.txt', 'r')
Lines = file1.readlines()
nahj = []
count = 1
i = 0
new_file = open(data_set_root + "light_nahj.txt", "w")

for line in Lines:
    splited = line.split("\t")
    i +=1
    if len(splited[1]) < 400:
      new_file. write(str(splited[0]) + "\t" + splited[1])
      # Nahj_table = Nahj_table.append({'output_text':splited[1],'vector':laser.embed_sentences(splited[1], 'ar'), 'output_data_source':'حکمت نهج البلاغه','index':i}, ignore_index = True)
file1.close()
new_file.close()


Nahj Al-Balaqa

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
Nahj_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'nahj/light_nahj.txt', 'r')
Lines = file1.readlines()
nahj = []
count = 1
i = 0
for line in Lines:
    splited = line.split("\t")
    i +=1
    Nahj_table = Nahj_table.append({'output_text':splited[1],'vector':laser.embed_sentences(splited[1], 'ar'), 'output_data_source':'حکمت نهج البلاغه','index':i}, ignore_index = True)
file1.close()


In [ ]:
pd.DataFrame(Nahj_table).to_csv(data_set_root+ "LASER_output/nahj.csv")

In [ ]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(Nahj_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(Nahj_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + Nahj_table["output_text"].iloc[index] )  
  print("_"*100) 
     

باید حقوق حیوانات را رعایت کنیم => وَ قَالَ ع احذَرُوا نِفَارَ النّعَمِ فَمَا کُلّ شَارِدٍ بِمَردُودٍ

____________________________________________________________________________________________________
نرسیدن به اهداف نباید باعث ناراحتی شود => أَمّا بَعدُ فَإِنّ المَرءَ قَد یَسُرّهُ دَرکُ مَا لَم یَکُن لِیَفُوتَهُ وَ یَسُوؤُهُ فَوتُ مَا لَم یَکُن لِیُدرِکَهُ فَلیَکُن سُرُورُکَ بِمَا نِلتَ مِن آخِرَتِکَ وَ لیَکُن أَسَفُکَ عَلَی مَا فَاتَکَ مِنهَا وَ مَا نِلتَ مِن دُنیَاکَ فَلَا تُکثِر بِهِ فَرَحاً وَ مَا فَاتَکَ مِنهَا فَلَا تَأسَ عَلَیهِ جَزَعاً وَ لیَکُن هَمّکَ فِیمَا بَعدَ المَوتِ

____________________________________________________________________________________________________
کنترل نکردن زبان انسان را بی‌ارزش می‌کند. => وَ اللّهِ مَا مُعَاوِیَةُ بِأَدهَی منِیّ وَ لَکِنّهُ یَغدِرُ وَ یَفجُرُ وَ لَو لَا کَرَاهِیَةُ الغَدرِ لَکُنتُ مِن أَدهَی النّاسِ وَ لَکِن کُلّ غُدَرَةٍ فُجَرَةٌ وَ کُلّ فُجَرَةٍ کُفَرَةٌ وَ لِکُلّ غَادِرٍ لِوَاءٌ یُعرَفُ بِهِ یَومَ القِیَامَةِ وَ اللّهِ مَا أُ

Yek-Beyti

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
tak_beit_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'poems/yek-beiti.txt', 'r')
Lines = file1.readlines()
i = 1
for line in Lines:
    tak_beit_table = tak_beit_table.append({'output_text':line,'vector':laser.embed_sentences(line, 'fa'),'output_data_source':" شعرهای تک بیتی",'index':i}, ignore_index = True)
    i+=1
file1.close()
     


In [ ]:
pd.DataFrame(tak_beit_table).to_csv(data_set_root+ "LASER_output/poems.csv")


*Evaluate* 

In [ ]:


for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(tak_beit_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(tak_beit_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + tak_beit_table["output_text"].iloc[index] )  
  print("_"*100) 
     


باید حقوق حیوانات را رعایت کنیم => قانع به یک استخوان چو کرکس بودن/ به که طفیل خوان ناکس بودن

____________________________________________________________________________________________________
نرسیدن به اهداف نباید باعث ناراحتی شود => آب طلب نکرده همیشه مراد نیست/ گاهی بهانه ای است که قربانی ات کنند

____________________________________________________________________________________________________
کنترل نکردن زبان انسان را بی‌ارزش می‌کند. => ز گفتار بیهوده جز رنج نیست/ چو خاموشی اندر جهان گنج نیست

____________________________________________________________________________________________________
انسان واقعی در برابر ظلم سکوت نمی‌کند => ز گفتار بیهود جز رنج نیست / چو خاموشی اندر جهان گنج نیست

____________________________________________________________________________________________________
باید به حقوق همسایگان خود احترام گذاشت => چو کم نور است چشمی بار عینک را کشد بینی/ ز بینی باید آموزی ره همسایه داری را

______________________________________________________________________

غزلیات حافظ







In [ ]:
columns = ["output_text" , "vector", " ","index"]
hafez_table = pd.DataFrame([], columns=columns)
file1 = open( data_set_root +'hafez/hafez-qazals.txt', 'r')
Lines = file1.readlines()
flag = False
qazals = []
i = 0
for line in Lines:
    qazal = r".*غزل.*"
    qazal_reg = f"{qazal}"    
    if(re.match(qazal_reg, line) and len(line)<20):
      print(line)
      input =  '\r\n'.join(qazals)
      i += 1
      hafez_table = hafez_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa'), 'output_data_source':'غزلیات حافظ', 'index':i}, ignore_index = True)
      qazals = []
    else:
      if(len(line)>5): 
        qazals.append(line.strip())

    
file1.close()

     


﻿غزل    ۱

غزل    ۲

غزل    ۳

غزل   ۴

غزل   ۵

غزل    ۶

غزل    ۷

غزل    ۸

غزل    ۹

غزل    ۱۰

غزل    ۱۰

غزل    ۱۲

غزل    ۱۳

غزل    ۱۴

غزل   ۱۵

غزل    ۱۶

غزل    ۱۷

غزل   ۱۸

غزل    ۱۹

غزل    ۲۰

غزل    ۲۱

غزل    ۲۲

غزل    ۲۳

غزل    ۲۴

غزل    ۲۵

غزل    ۲۶

غزل    ۲۷

غزل    ۲۸

غزل    ۲۹

غزل    ۳۰

غزل    ۳۱

غزل    ۳۲

غزل    ۳۳

غزل   ۳۴

غزل    ۳۵

غزل    ۳۶

غزل    ۳۷

غزل    ۳۸

غزل    ۳۹

غزل    ۴۰

غزل    ۴۱

غزل    ۴۲

غزل    ۴۳

غزل    ۴۴

غزل    ۴۵

غزل    ۴۶

غزل    ۴۷

غزل    ۴۸

غزل    ۴۹

غزل    ۵۰

غزل    ۵۱

غزل    ۵۲

غزل    ۵۳

غزل    ۵۴

غزل    ۵۵

غزل    ۵۶

غزل    ۵۷

غزل    ۵۸

غزل    ۵۹

غزل    ۶۰

غزل    ۶۱

غزل    ۶۲

غزل    ۶۳

غزل    ۶۴

غزل    ۶۵

غزل    ۶۶

غزل    ۶۷

غزل    ۶۸

غزل    ۶۹

غزل    ۷۰

غزل    ۷۱

غزل    ۷۲

غزل    ۷۳

غزل    ۷۴

غزل    ۷۵

غزل    ۷۶

غزل    ۷۷

غزل    ۷۸

غزل    ۷۹

غزل    ۸۰

غزل    ۸۱

غزل    ۸۲

غزل    ۸۳

غزل    ۸۴

غزل    ۸۵

غزل    ۸۶

غزل    ۸۷

غزل    ۸۸

غزل    ۸۹

غزل    ۹۰

غزل    ۹۱

غزل    ۹۲

غ

In [ ]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(hafez_table)-1):
    image = np.array(input_table["vector"].iloc[k])@np.array(hafez_table["vector"].iloc[i+1]).T
    if(image > max):
      max = image
      index = i+1

  print(input_table["input_text"].iloc[k] + "  ======>>>> " + " غزل شماره " + str(index))  
  print(hafez_table["output_text"].iloc[index] )
  print("_"*100) 

باید حقوق حیوانات را رعایت کنیم  ======>>>>  غزل شماره 184
دوش ديدم که ملايک در ميخانه زدند
گل آدم بسرشتند و به پيمانه زدند
ساکنان حرم ستر و عفاف ملکوت
با من راه نشين باده مستانه زدند
آسمان بار امانت نتوانست کشيد
قرعه کار به نام من ديوانه زدند
جنگ هفتاد و دو ملت همه را عذر بنه
چون نديدند حقيقت ره افسانه زدند
شکر ايزد که ميان من و او صلح افتاد
صوفيان رقص کنان ساغر شکرانه زدند
آتش آن نيست که از شعله او خندد شمع
آتش آن است که در خرمن پروانه زدند
کس چو حافظ نگشاد از رخ انديشه نقاب
تا سر زلف سخن را به قلم شانه زدند
____________________________________________________________________________________________________
نرسیدن به اهداف نباید باعث ناراحتی شود  ======>>>>  غزل شماره 378
ما نگوييم بد و ميل به ناحق نکنيم
جامه کس سيه و دلق خود ازرق نکنيم
عيب درويش و توانگر به کم و بيش بد است
کار بد مصلحت آن است که مطلق نکنيم
رقم مغلطه بر دفتر دانش نزنيم
سر حق بر ورق شعبده ملحق نکنيم
شاه اگر جرعه رندان نه به حرمت نوشد
التفاتش به می صاف مروق نکنيم
خوش برانيم جهان در نظر راهروان
فکر اسب سيه و زين مغرق نک

In [ ]:
pd.DataFrame(hafez_table).to_csv(data_set_root+ "LASER_output/hafez.csv")

غزلیات شمس

In [ ]:
moulavi_qazals_path = data_set_root + "moulavi/ultra_light_divane-e-shams.txt"
moulavi_qazals_pattern = re.compile(r'\n?\s*\d+\s*?\n')

def read_divan_shams():
    with open(moulavi_qazals_path, encoding='utf-8') as f:
        text = f.read()
        
    qazals_arr = moulavi_qazals_pattern.split(text)
    return qazals_arr[1:]


Creat Ultra light wight and ligthweight dataset

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
molavi_table = pd.DataFrame([], columns=columns)
molavi_data_len = len(read_divan_shams())
input_set = read_divan_shams()
new_file = open(data_set_root + "moulavi/light_divane-e-shams.txt", "w")
for i in range(0, molavi_data_len):
  if len(input_set[i]) < 600:
    input = input_set[i]
    new_file. write(str(i) + '\n'+ input + '\n\n')
new_file.close()
    #molavi_table = molavi_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa'), 'output_data_source':'غزلیات مولوی', 'index': i+1}, ignore_index = True)

In [ ]:
columns = ["output_text" , "vector", "output_data_source","index"]
molavi_table = pd.DataFrame([], columns=columns)
molavi_data_len = len(read_divan_shams())
input_set = read_divan_shams()

for i in range(0, molavi_data_len):
    input = input_set[i]
    molavi_table = molavi_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa'), 'output_data_source':'غزلیات مولوی', 'index': i+1}, ignore_index = True)

In [ ]:
print(molavi_table)

                                           output_text  \
0    ای خواجه نمی بینی این روز قیامت را\tای خواجه ن...   
1    شاد آمدی ای مه رو ای شادی جان شاد آ\tتا بود چن...   
2    یک پند ز من بشنو خواهی نشوی رسوا\tمن خمره افیو...   
3    میندیش میندیش که اندیشه گری ها\tچو نفطند بسوزن...   
4    چو او باشد دل دلسوز ما را\tچه باشد شب چه باشد ...   
..                                                 ...   
330  املا قدح البقا ندیمی!\tمن خمرة دنک القدیم\nصحی...   
331  طارت حیلی و زال حیلی\tاصبحت مکابدا لویلی\nقد ا...   
332  قالت الکاس ارفعونی کم تحبسونی\tان جسمی فی زجاج...   
333  الا حریم لیلی، علیکم سلامی\tادرتم علینا صفیة ا...   
334  قلت له مصیحا یا ملک المشرق\tاقسم بالخالق مثلک ...   

                                                vector output_data_source  \
0    [[0.0039898534, -4.7282656e-06, 0.00090174546,...       غزلیات مولوی   
1    [[0.005033174, 0.0009092737, 0.00028622386, 0....       غزلیات مولوی   
2    [[0.0010444579, 0.0023202559, 0.009110144, 0.0...       غزلیات مولو

In [ ]:
# note that these are only the poems that their length is up tp 800 character
pd.DataFrame(molavi_table).to_csv(data_set_root+ "LASER_output/moulavi_800.csv")

In [ ]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(molavi_table)-1):
    image = np.array(input_table["vector"].iloc[k])@np.array(molavi_table["vector"].iloc[i+1]).T
    if(image > max):
      max = image
      index = i+1

  print(input_table["input_text"].iloc[k] + "  ======>>>> ")  
  print(molavi_table["output_text"].iloc[index] )
  print("_"*100) 

باید حقوق حیوانات را رعایت کنیم  ======>>>> 
علم عشق برآمد برهانم ز زحیرم	به لب چشمه حیوان بکشم پای بمیرم
به که مانم به که مانم که سطرلاب جهانم	چو قضا حکم روانم نه امیرم نه وزیرم
بروی ای عالم هستی همه را پای ببستی	تو اگر جان منستی نپذیرم نپذیرم
____________________________________________________________________________________________________
نرسیدن به اهداف نباید باعث ناراحتی شود  ======>>>> 
زان شاه که او را هوس طبل و علم نیست	دیوانه شدم بر سر دیوانه قلم نیست
از دور ببینی تو مرا شخص رونده	آن شخص خیالست ولی غیر عدم نیست
پیش آ و عدم شو که عدم معدن جانست	اما نه چنین جان که بجز غصه و غم نیست
من بی من و تو بی تو درآییم در این جو	زیرا که در این خشک بجز ظلم و ستم نیست
این جوی کند غرقه ولیکن نکشد مرد	کو آب حیاتست و بجز لطف و کرم نیست
____________________________________________________________________________________________________
کنترل نکردن زبان انسان را بی‌ارزش می‌کند.  ======>>>> 
زان شاه که او را هوس طبل و علم نیست	دیوانه شدم بر سر دیوانه قلم نیست
از دور ببینی تو مرا شخص رونده	آن شخص

 تجمیع دادگان خروجی

**Selecting Top N Output for each input**

In [ ]:


def create_pair_knn(k,model_name,output_data_source,input_table,output_table):
  columns = ["model","input_text","output_data_source","input_vector","output_text" , "output_vector"]
  pair_table = pd.DataFrame([], columns=columns)
  for i in range(len(input_table)):
    vector = []
    for j in range(len(output_table)): 
      input_vector = input_table.iloc[i]['vector']
      output_vector = output_table.iloc[j]['vector']
      vector.append(abs(input_vector[0] @ output_vector[0].T))
    vector = np.array(vector)
    indexes = vector.argsort()[-k:]
    
    for index in indexes:
      output_data_source_index = str(output_data_source) + " => " + str(index + 1)
      pair_table = pair_table.append({'model':model_name,"output_data_source":output_data_source_index,"input_text":input_table.iloc[i]['input_text'],"input_vector":input_table.iloc[i]['vector'],"output_text":output_table.iloc[index]['output_text'] , "output_vector":output_table.iloc[index]['vector']}, ignore_index = True)
  
  return pair_table

     


In [ ]:
create_pair_knn(3,"LASER","تک ابیات",input_table, tak_beit_table)

,model,input_text,output_data_source,input_vector,output_text,output_vector
0,LASER,باید حقوق حیوانات را رعایت کنیم,تک ابیات => 159,"[[0.0018938975, -6.682478e-05, 0.0017122207, 0...",آنجا که جهل مایه تمکین و سروریست/ باید به روز ...,"[[0.00038259168, 0.022177326, 0.0001327116, 0...."
1,LASER,باید حقوق حیوانات را رعایت کنیم,تک ابیات => 285,"[[0.0018938975, -6.682478e-05, 0.0017122207, 0...",جایی که بود شمع رخت انجمن افروز/ خورشید چراغیس...,"[[0.0036351224, 0.011773068, 0.0013317887, 0.0..."
2,LASER,باید حقوق حیوانات را رعایت کنیم,تک ابیات => 247,"[[0.0018938975, -6.682478e-05, 0.0017122207, 0...",قانع به یک استخوان چو کرکس بودن/ به که طفیل خو...,"[[0.007646164, 0.002562769, 0.00035679742, 0.0..."
3,LASER,نرسیدن به اهداف نباید باعث ناراحتی شود,تک ابیات => 10,"[[0.0050546504, 0.0023843201, 0.0010176041, 0....",ز گفتار بیهوده جز رنج نیست/ چو خاموشی اندر جها...,"[[0.018255733, -0.000117060714, -0.0030013558,..."
4,LASER,نرسیدن به اهداف نباید باعث ناراحتی شود,تک ابیات => 332,"[[0.0050546504, 0.0023843201, 0.0010176041, 0....",خاطر آزرده چه داری ز غم سود و زیان/ غم مخور، ش...,"[[-0.00031172144, 0.012453295, -0.001193569, 0..."
...,...,...,...,...,...,...
295,LASER,صبر یکی از صفات خردمندان است,تک ابیات => 247,"[[0.014067072, -0.00011483547, -0.0008464994, ...",قانع به یک استخوان چو کرکس بودن/ به که طفیل خو...,"[[0.007646164, 0.002562769, 0.00035679742, 0.0..."
296,LASER,صبر یکی از صفات خردمندان است,تک ابیات => 159,"[[0.014067072, -0.00011483547, -0.0008464994, ...",آنجا که جهل مایه تمکین و سروریست/ باید به روز ...,"[[0.00038259168, 0.022177326, 0.0001327116, 0...."
297,LASER,ارزشمندترین دوستی، دوستی در راه خداست,تک ابیات => 94,"[[0.0045173448, 2.8629329e-05, 0.00014082856, ...",خداوندا حضوری بخش ما را/ اجابت کن به فضلت این ...,"[[0.007120842, -0.00012871508, 0.0011519053, 0..."
298,LASER,ارزشمندترین دوستی، دوستی در راه خداست,تک ابیات => 201,"[[0.0045173448, 2.8629329e-05, 0.00014082856, ...",قفس شکسته و بال و پرم گشاده و ترسم/ خدا نکرده ...,"[[0.001582338, -2.3269453e-05, 0.011936702, 0...."


Cosine KNN

In [ ]:
def create_pair_cosine_knn(k,model_name,output_data_source,input_table,output_table):
  columns = ["model","input_text","output_data_source","input_vector","output_text" , "output_vector"]
  pair_table = pd.DataFrame([], columns=columns)
  model = NearestNeighbors(n_neighbors=k,
                          metric='cosine',
                          algorithm='brute',
                          n_jobs=-1)
  x = []
  # Convert out put to array
  for i in range (0, len(tak_beit_table)):
    x.append(tak_beit_table["vector"][i][0])

  model.fit(x)

  for n in range(0, len(input_table)):
    v = input_table["vector"][n]
    for j in range(0, k):
      output_data_source_index = str(model.kneighbors(v, k, return_distance=False)[0][j])
      output_data_source_compelete = str(output_data_source) + " => " + output_data_source_index
      input_text = input_table['input_text'][n]
      output_text = tak_beit_table["output_text"][model.kneighbors(v, k, return_distance=False)[0][j]]
      output_vector = tak_beit_table["vector"][model.kneighbors(v, k, return_distance=False)[0][j]]
      pair_table = pair_table.append({'model':model_name,"output_data_source":output_data_source_compelete,"input_text":input_text,"input_vector":v ,"output_text":output_text , "output_vector":output_vector}, ignore_index = True)
  return pair_table


In [ ]:
create_pair_cosine_knn(3,"LASER","تک ابیات",input_table, tak_beit_table)

,model,input_text,output_data_source,input_vector,output_text,output_vector
0,LASER,باید حقوق حیوانات را رعایت کنیم,تک ابیات => 240,"[[0.0018938975, -6.682478e-05, 0.0017122207, 0...",اجرام که ساکنان این ایوانند/ اسباب تردد خردمند...,"[[0.0041127848, 0.0034398495, 0.0010181738, 0...."
1,LASER,باید حقوق حیوانات را رعایت کنیم,تک ابیات => 338,"[[0.0018938975, -6.682478e-05, 0.0017122207, 0...",خود سازی پیران بود افزون ز جوانان/ تعمیر ضرورس...,"[[0.0018850673, -0.00011026716, 0.0022377928, ..."
2,LASER,باید حقوق حیوانات را رعایت کنیم,تک ابیات => 223,"[[0.0018938975, -6.682478e-05, 0.0017122207, 0...",از طلا گشتن پشیمان گشته ایم/ مرحمت فرموده ما ر...,"[[-2.0967494e-05, 0.0016264333, 0.00044711735,..."
3,LASER,نرسیدن به اهداف نباید باعث ناراحتی شود,تک ابیات => 230,"[[0.0050546504, 0.0023843201, 0.0010176041, 0....",امیر قافله را هم تغافلی باید/ که نا‌امید نگردن...,"[[0.000115753464, -0.00019203873, 0.0006668653..."
4,LASER,نرسیدن به اهداف نباید باعث ناراحتی شود,تک ابیات => 44,"[[0.0050546504, 0.0023843201, 0.0010176041, 0....",کوه ناهموار را هموار کردن سخت نیست/ حرف نا همو...,"[[5.1379353e-05, -0.00023895215, 0.0034602594,..."
...,...,...,...,...,...,...
295,LASER,صبر یکی از صفات خردمندان است,تک ابیات => 335,"[[0.014067072, -0.00011483547, -0.0008464994, ...",خموشی پیشه کن در مکتب عشق/ که درس عاشقی بی قیل...,"[[0.004498319, 0.0022696138, -2.045887e-05, 0...."
296,LASER,صبر یکی از صفات خردمندان است,تک ابیات => 50,"[[0.014067072, -0.00011483547, -0.0008464994, ...",دل که رنجید از کسی خرسند کردن مشکل است/ شیشه ب...,"[[0.0145353805, -0.00024047811, -0.00075667474..."
297,LASER,ارزشمندترین دوستی، دوستی در راه خداست,تک ابیات => 79,"[[0.0045173448, 2.8629329e-05, 0.00014082856, ...",غمگین نیم که خلق شمارند بد مرا/ نزدیک می‌کند ب...,"[[-0.0013371883, -5.69236e-06, 0.0012265603, 0..."
298,LASER,ارزشمندترین دوستی، دوستی در راه خداست,تک ابیات => 301,"[[0.0045173448, 2.8629329e-05, 0.00014082856, ...",چشم امید بر کرم حضرت جلال/ چون گوش روزده دار ب...,"[[0.01111768, -5.759189e-05, 0.00025559362, 0...."
